In [1]:
####### Usage Instructions #######

### FMindex
# Enter text string into  FMindex

### Important attributes
# charList = array - alphabetically sorted unique characters within text including $
# charSet = dictionary - all unique characters and their number of occurences
# suffixArray = array
# firstCol = string - characters of the first column of the BWT matrix
# lastCol = string - BWT
# skip = dictionary - all unique characters and total number of characters that occur before it alphabetically
# LF = array - LF mapping
# fArray = array with n number of subarrays where n is number of unique characters - subarray is indexed based on 
#           order it appears in charList, each subarray contains number of occurences of that character in 
#           firstCol[:i] (including i)
# lArray = array with n number of subarrays where n is number of unique characters - subarray is indexed based on 
#           order it appears in charList, each subarray contains number of occurences of that character in  
#           lastCol[:i] (including i)

# To match, call function substringMatch(pattern)


### KMP
# Enter pattern into KMP
# To match, call function substringMatch(text, pattern)


# Text and Pattern should only contain the alphabet, no unique characters are allowed



In [2]:
import time
start_time = time.time()


class FMindex():
    
    
    def __init__(self, text):
        '''
        Creates suffix array in O(nlogn) time 
        From suffix array, obtain first column and second column of the BWT
        Create LF array
        substringMatching produces index where match occurs
        '''
        
        text = text.upper()
        if text[-1] != '$':
            text += '$'
        
        self.charList, self.charSet = self.charDict(text)
        print("--- %s seconds .1. ---" % (time.time() - start_time))
        
        self.suffixArray = self.suffixArray(text, self.charList)
        print("--- %s seconds .1. ---" % (time.time() - start_time))
        
        self.firstCol, self.lastCol = self.firstLast(text, self.suffixArray)
        print("--- %s seconds .1. ---" % (time.time() - start_time))
        
        self.skip = self.skip(self.charList, self.charSet)
        print("--- %s seconds .1. ---" % (time.time() - start_time))
        
        self.LF, self.fArray, self.lArray = self.LF(self.charList, self.firstCol, self.lastCol)
        print("--- %s seconds .1. ---" % (time.time() - start_time))

        
    def charDict(self, text):
        '''
        Runs through text to find all unique characters and counts
        
        Input: 
            text 
            
        Output: 
            charlist = array of all unique characters in alphabetical order
            charset = dictionary with unique characters as keys and number of occurences as values
        '''
        charset = dict()
        for i in range(len(text)):
            charset[text[i]] = charset.get(text[i],0) + 1
        charlist = ''.join(charset.keys())
        charlist = sorted(charlist)
        
        return charlist, charset


    def suffixArray(self, text, charlist):
        '''
        Create suffix array in O(nlogn) time
        Takes alphabetically ordered unique characters in charlist and assigns values starting at 1 
            (0 is reserved for out of bounds of array during sorting step)
        Assigns ranking to all characters in the text 
        Sort suffixes according to the first 2**i characters starting at i = 0 until 2**i > len(text) or 
            until all ranking is unique
        Returns suffix array
        
        Input: 
            text 
            charlist
            
        Output: 
            answer = array containing indexes of suffixes in alphabetical order (suffix array )
        
        '''
        ranking = []
        for i in range(0,len(charlist)):
            ranking.append(i+1)   # since adding $
        
        SA = []
        maxNum = 0
        for i in range(0,len(text)):
            SA.append(ranking[charlist.index(text[i])])
            maxNum = max(maxNum, ranking[charlist.index(text[i])])
        
        j = 0
        out = 0

        while 2**j<len(text) and maxNum != len(SA)-1:
            tens = SA
            ones = []
            
            for i in range(0,len(SA)):
                if i + 2**j >= len(text):
                    ones.append(0)
                else:
                    ones.append(SA[i+2**j])            
            j+= 1
            
            for m in reversed(range(0,2)):
                if m == 1:
                    firstOrderedOnes = [0]*(len(SA)+1)
                    firstOrderedTens = [0]*(len(SA)+1)
                    count = [0]*(maxNum+1)
                
                    for i in range(0,len(SA)):
                        count[ones[i]] += 1
                    
                    for i in range(1,len(count)):
                        count[i] += count[i-1] 
                    
                    for i in reversed(range(0,len(SA))):
                        index = count[ones[i]] 
                        firstOrderedOnes[index] = ones[i]
                        firstOrderedTens[index] = tens[i]
                        count[ones[i]] -= 1
                    firstOrderedTens = firstOrderedTens[1:]
                    firstOrderedOnes = firstOrderedOnes[1:]
            
                else:
                    finalTens = [0]*(len(SA)+1)
                    finalOnes = [0]*(len(SA)+1)
                    count = [0]*(maxNum+1)
                    
                    for i in range(0,len(SA)):
                        count[firstOrderedTens[i]] += 1
                    
                    for i in range(1,len(count)):
                        count[i] += count[i-1] 
                
                    for i in reversed(range(0,len(SA))):
                        index = count[firstOrderedTens[i]]
                        finalOnes[index] = firstOrderedOnes[i]
                        finalTens[index] = firstOrderedTens[i]
                        count[firstOrderedTens[i]] -=1
                    finalTens = finalTens[1:]
                    finalOnes = finalOnes[1:]
            
            dictionary = dict()
            key = 0
            for i in range(0,len(SA)):
                if str([finalTens[i],finalOnes[i]]) not in dictionary:
                    dictionary[str([finalTens[i],finalOnes[i]])] = key
                    key += 1
                else:
                    continue
                
            updatedSA = [0]*len(SA)
            for i in range(0,len(SA)):
                updatedSA[i] = dictionary[str([SA[i],ones[i]])]

            SA = updatedSA
            maxNum = key-1
            
        answer = [0]*len(text)
        for i in range(0,len(SA)):
            answer[SA[i]] = i

        return answer

    
    def firstLast(self, text, suffixarray):
        '''
        From suffix array, obtain the first and last column of the BWT array
        
        Input: 
            text 
            suffixarray
            
        Output: 
            first = string of first column of BWT array
            last = string of last column of BWT array (just the BWT itself)
        '''
        first = ""
        last = ""

        for i in range(0,len(suffixarray)):
            first += text[suffixarray[i]]
            last += text[-1] if suffixarray[i] == 0 else text[suffixarray[i]-1]
        
        return first, last
    
    
    def skip(self, charlist, charset):
        '''
        For each unique character, calculate the number of characters that appear before it alphabetically
        EX: AABDCC$
        Dict = {'$'= 0, 'A'= 1, 'B'= 3, 'C'=4, 'D'=6}
        
        Input: 
            charlist 
            charset
            
        Output: 
            skiplist = dictionary where key is unique character and value is the number of characters 
                       that appear before it alphabetically
            '''
        total = 0
        skiplist = dict()
        for i in range(len(charlist)):
            temp = charset[charlist[i]]
            skiplist[charlist[i]] = total
            total += temp
        
        return skiplist
        
        
    def select(self, array, num):
        '''
        Finds the first index where num is found
        EX: array = [0,0,0,1,2,3,4,4,4,5], num = 4 -> output = 6
        
        
        Input: 
            array = number of instances a character shows up in the text from text[:i]
            num = number wanted to be found
            
        Output: 
            mid = index where the the first instance of num is found
        '''
        if array[0] == num:
            return 0
        
        l = 0
        r = len(array) - 1
        
        while l <= r:
            mid = (l + r)//2
            if array[mid] == num and array[mid-1] == num-1:
                return mid
            if array[mid] < num:
                l = mid +1
            else:
                r = mid-1


    def LF(self, charlist, firstcol, lastcol):
        '''
        returns LF indexing
        
        
        Input: 
            charlist
            firstcol = first column of BWT array
            lastcol = last column of BWT array
            
        Output: 
            LF = array that maps the last column of BWT array to first column of BWT array 
            farray = array in array for firstcol
                        index to subarray  matches alphabetical order of unique characters
                        length of each subarray = length of text, number of instances a character shows up in 
                            the text from text[:i]
            larray = array in array for lastcol
                        index to subarray  matches alphabetical order of unique characters
                        length of each subarray = length of text, number of instances a character shows up in 
                            the text from text[:i]
        '''
        farray = [[]  for i in range(len(charlist))]
        larray = [[] for i in range(len(charlist))]
        
        for i in range(len(firstcol)):
            first = firstcol[i]
            last = lastcol[i]
            for j in range(len(charlist)):
                if first == charlist[j]:
                    farray[j].append(1) if i == 0 else farray[j].append(farray[j][-1]+1)
                else:
                    farray[j].append(0) if i == 0 else farray[j].append(farray[j][-1])
        
                if last == charlist[j]:
                    larray[j].append(1) if i == 0 else larray[j].append(larray[j][-1]+1)
                else:
                    larray[j].append(0) if i == 0 else larray[j].append(larray[j][-1])

        row = 0
        answer = ""
        LF = []
        for i in range(len(firstcol)):
            index = charlist.index(lastcol[i])
            rank = larray[index][i]
            row = self.select(farray[index],rank)
            LF.append(row) if row else LF.append(0)        

        return LF, farray, larray
    
    
    def substringMatch(self, pattern):
        '''
        Searches through the text to find the index of pattern (only alphabetical characters)
        Finds pattern in reverse order
        top and bottom - all rows inbetween adn including top and bottom are a match
        firstOccurence and lastOccurence - matches the next index of the pattern (in reversed order)
        
        Input: 
            pattern = substring user wants to find in the text (only alphabetical characters)
            
        Output: 
            sorted(answer) = indexes where the pattern can be found in the text, will return empty array if not found
        '''
        pattern = pattern.upper()
        
        if len(pattern) == 0:
            return []
        
        i = len(pattern)-1
        if pattern[i] not in self.charSet or pattern[i] == "$":
                return []

        index = self.charList.index(pattern[i])
        top = self.select(self.fArray[index],1)
        
        bottom = self.skip[self.charList[index+1]]-1 if index+1 < len(self.charList) else len(self.fArray[index])-1

        i -= 1
        while i >= 0 and top <= bottom:
            if pattern[i] not in self.charSet:
                return []
            index = self.charList.index(pattern[i])
            firstOccurence = self.lArray[index][top]
            if firstOccurence == 0:
                firstOccurence = 1
            elif firstOccurence == self.lArray[index][top-1]:
                firstOccurence += 1
            lastOccurence = self.lArray[index][bottom]
    
            top = self.skip[pattern[i]]+firstOccurence-1
            bottom = self.skip[pattern[i]]+lastOccurence-1
            i -= 1
            
            
        answer = []
        for i in range(top, bottom+1):
            answer.append(self.suffixArray[i])
            
        return sorted(answer)

In [3]:
class KMP():
    '''
    Takes in a pattern
    Can use substringMatch to find indices where pattern appears in text
    '''
    
    def __init__(self, pattern):
        pattern = pattern.upper()
        '''
        create pi array and stores in self.pi
        '''
        self.pi = self.pi(pattern)
        
    def pi(self, pattern):
        '''
        Creates the pi array
        '''
        pi = [0]*len(pattern)
        for i in range(1,len(pattern)):
            b = pi[i-1]
            while b > 0 and pattern[b] != pattern[i]:
                b = pi[b-1]
            if pattern[b] == pattern[i]:
                b += 1
            pi[i] = b
        return pi
        
        
    def substringMatch(self, text, pattern):
        '''
        Finds indices where pattern occur in the text
        '''
        if len(pattern) == 0:
            return []
        
        text = text.upper()
        pattern = pattern.upper()
        
        index = []
        i = 0
        l = 0
        while i < len(text) - len(pattern) + 1:
            while l < len(text) and l < len(pattern) and text[i+l] == pattern[l]:
                l += 1
            if l == len(pattern):
                index.append(i)
        
            if l > 0:
                i += (l - self.pi[l-1])
                l = self.pi[l-1]
            else:
                i += 1
        return index

In [4]:
# Sample text and test cases
a = "mississippi"
b = ""              # empty string
c = "missis"        # match at 0
d = "issippi"       # match at middle
e = "ippi"          # match at end
f = "issi"          # 2 matches
g = "$"             # $ character in text
h = "i"             # character in text
i = "m"             # character in text
j = "p"             # character in text
k = "s"             # character in text
l = "a"             # character NOT in text
m = "xmississippi"  # pattern is longer by addition at the front
n = "mississippix"  # pattern is longer by addition at the end
o = "missisxsippi"  # pattern is longer by addition at the middle
p = "issii"         # pattern not found (mismatch at the end)
q = "iissi"         # pattern not found (mismatch at the front)
r = "isssi"         # pattern not found (mismatch at the middle)
s = "iissii"        # pattern not found (mismatch at front and end)
t = "xississippi"   # pattern same length (mismatch at the front)
u = "mississippx"   # pattern same length (mismatch at the end)
v = "missixsippi"   # pattern same length (mismatch at the middle)

In [5]:
start_time = time.time()
fm = FMindex(a)
print('\n')

print('charList')
print(fm.charList)
print('\n')

print('charSet')
print(fm.charSet)
print('\n')

print('suffixArray')
print(fm.suffixArray)
print('\n')

print('firstCol')
print(fm.firstCol) 
print('\n')

print('lastCol')
print(fm.lastCol)
print('\n')

print('skip')
print(fm.skip) 
print('\n')

print('LF')
print(fm.LF) 
print('\n')

print('fArray')
print(fm.fArray) 
print('\n')

print('lArray')
print(fm.lArray)
print('\n')

print('Starting index where pattern matches to text')
print(a + ': ' + str(fm.substringMatch(a)))
print(b + ': ' + str(fm.substringMatch(b)))
print(c + ': ' + str(fm.substringMatch(c)))
print(d + ': ' + str(fm.substringMatch(d)))
print(e + ': ' + str(fm.substringMatch(e)))
print(f + ': ' + str(fm.substringMatch(f)))
print(g + ': ' + str(fm.substringMatch(g)))
print(h + ': ' + str(fm.substringMatch(h)))
print(i + ': ' + str(fm.substringMatch(i)))
print(j + ': ' + str(fm.substringMatch(j)))
print(k + ': ' + str(fm.substringMatch(k)))
print(l + ': ' + str(fm.substringMatch(l)))
print(m + ': ' + str(fm.substringMatch(m)))
print(n + ': ' + str(fm.substringMatch(n)))
print(o + ': ' + str(fm.substringMatch(o)))
print(p + ': ' + str(fm.substringMatch(p)))
print(q + ': ' + str(fm.substringMatch(q)))
print(r + ': ' + str(fm.substringMatch(r)))
print(s + ': ' + str(fm.substringMatch(s)))
print(t + ': ' + str(fm.substringMatch(t)))
print(u + ': ' + str(fm.substringMatch(u)))
print(v + ': ' + str(fm.substringMatch(v)))

--- 3.719329833984375e-05 seconds .1. ---
--- 0.0004851818084716797 seconds .1. ---
--- 0.0005140304565429688 seconds .1. ---
--- 0.0005340576171875 seconds .1. ---
--- 0.0006022453308105469 seconds .1. ---


charList
['$', 'I', 'M', 'P', 'S']


charSet
{'M': 1, 'I': 4, 'S': 4, 'P': 2, '$': 1}


suffixArray
[11, 10, 7, 4, 1, 0, 9, 8, 6, 3, 5, 2]


firstCol
$IIIIMPPSSSS


lastCol
IPSSM$PISSII


skip
{'$': 0, 'I': 1, 'M': 5, 'P': 6, 'S': 8}


LF
[1, 6, 8, 9, 5, 0, 7, 2, 10, 11, 3, 4]


fArray
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2], [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4]]


lArray
[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 4], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2], [0, 0, 1, 2, 2, 2, 2, 2, 3, 4, 4, 4]]


Starting index where pattern matches to text
mississippi: [0]
: []
missis: [0]
issippi: [4]
ippi: [7]
issi: [1,

In [6]:
print('Starting index where pattern matches to text')
kmp = KMP(a)
print(a + ': ' + str(kmp.substringMatch(a, a)))
kmp = KMP(b)
print(b + ': ' + str(kmp.substringMatch(a, b)))
kmp = KMP(c)
print(c + ': ' + str(kmp.substringMatch(a, c)))
kmp = KMP(d)
print(d + ': ' + str(kmp.substringMatch(a, d)))
kmp = KMP (e)
print(e + ': ' + str(kmp.substringMatch(a, e)))
kmp = KMP (f)
print(f + ': ' + str(kmp.substringMatch(a, f)))
kmp = KMP (g)
print(g + ': ' + str(kmp.substringMatch(a, g)))
kmp = KMP (h)
print(h + ': ' + str(kmp.substringMatch(a, h)))
kmp = KMP (i)
print(i + ': ' + str(kmp.substringMatch(a, i)))
kmp = KMP (j)
print(j + ': ' + str(kmp.substringMatch(a, j)))
kmp = KMP (k)
print(k + ': ' + str(kmp.substringMatch(a, k)))
kmp = KMP (l)
print(l + ': ' + str(kmp.substringMatch(a, l)))
kmp = KMP (m)
print(m + ': ' + str(kmp.substringMatch(a, m)))
kmp = KMP (n)
print(n + ': ' + str(kmp.substringMatch(a, n)))
kmp = KMP (o)
print(o + ': ' + str(kmp.substringMatch(a, o)))
kmp = KMP (p)
print(p + ': ' + str(kmp.substringMatch(a, p)))
kmp = KMP (q)
print(q + ': ' + str(kmp.substringMatch(a, q)))
kmp = KMP (r)
print(r + ': ' + str(kmp.substringMatch(a, r)))
kmp = KMP (s)
print(s + ': ' + str(kmp.substringMatch(a, s)))
kmp = KMP (t)
print(t + ': ' + str(kmp.substringMatch(a, t)))
kmp = KMP (u)
print(u + ': ' + str(kmp.substringMatch(a, u)))
kmp = KMP (v)
print(v + ': ' + str(kmp.substringMatch(a, v)))

Starting index where pattern matches to text
mississippi: [0]
: []
missis: [0]
issippi: [4]
ippi: [7]
issi: [1, 4]
$: []
i: [1, 4, 7, 10]
m: [0]
p: [8, 9]
s: [2, 3, 5, 6]
a: []
xmississippi: []
mississippix: []
missisxsippi: []
issii: []
iissi: []
isssi: []
iissii: []
xississippi: []
mississippx: []
missixsippi: []


In [7]:
# Other texts to try

In [8]:
start_time = time.time()
alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
fm = FMindex(alpha)
print(fm.charList)
print(fm.charSet)
print(fm.suffixArray)
print(fm.firstCol) 
print(fm.lastCol)
print(fm.skip) 
print(fm.LF) 
print(fm.fArray) 
print(fm.lArray)

--- 5.91278076171875e-05 seconds .1. ---
--- 0.00022220611572265625 seconds .1. ---
--- 0.000244140625 seconds .1. ---
--- 0.00026297569274902344 seconds .1. ---
--- 0.0006098747253417969 seconds .1. ---
['$', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
{'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F': 1, 'G': 1, 'H': 1, 'I': 1, 'J': 1, 'K': 1, 'L': 1, 'M': 1, 'N': 1, 'O': 1, 'P': 1, 'Q': 1, 'R': 1, 'S': 1, 'T': 1, 'U': 1, 'V': 1, 'W': 1, 'X': 1, 'Y': 1, 'Z': 1, '$': 1}
[26, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
$ABCDEFGHIJKLMNOPQRSTUVWXYZ
Z$ABCDEFGHIJKLMNOPQRSTUVWXY
{'$': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}
[26, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,

In [9]:
start_time = time.time()

repeata = "aaaaaaaaaaaaaaaaaaaa" # a repeated 20 times
aa = "aaaaaaaaaaaa" # a repeated 12 times, this matches 9 times

fm = FMindex(repeata)
print(fm.substringMatch(aa))
print(fm.substringMatch("a"))

--- 6.29425048828125e-05 seconds .1. ---
--- 0.0004169940948486328 seconds .1. ---
--- 0.00043964385986328125 seconds .1. ---
--- 0.00045371055603027344 seconds .1. ---
--- 0.0005178451538085938 seconds .1. ---
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
